In [ ]:
import numpy as np
import scipy
import torch
from torch import nn
from torch import optim
import torchvision
from torchvision import datasets
import torchvision.transforms as transforms
from torch.utils.tensorboard import SummaryWriter
import matplotlib.pyplot as plt

# Manipulate the device and the precision

**Question**

In PyTorch, each tensor has a `device` and a `dtype`. The `device` refers to the device on which the tensor is currently loaded. The `dtype` is the type of data stored in the tensor. By default, `device = torch.device("cpu")` and `dtype = torch.float32`.

One can build a tensor directly on a given device and with a given data type (or precision) by using the optional arguments `device` and `dtype`, or we can build them first with by-default values, and send copy them on the desired device with the desired precision with the `to` method.

Use both methods with devices `torch.device("cuda")` and `torch.device("cpu")` and with data types other than `torch.float32`.

In [ ]:
with_cuda = torch.cuda.is_available()
if with_cuda:
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

# Backpropagation of the gradient

## Introduction

Code libraries such as PyTorch, TensorFlow, JAX implement **automatic differentation (AutoDiff)** methods, which are at the core of ML methods trainable by gradient-based optimization techniques. These ML method include naturally neural networks.

AutoDiff is based on the construction of a computational graph, which is a *Directed Acyclic Graph* (DAG). This graph represents the different stages of computation of a function $f$ at a point $\boldsymbol{\theta} = (\theta_1, \theta_2, \cdots, \theta_S)$, and we want to compute: 
$$
\frac{\partial f}{\partial \theta_1}, \frac{\partial f}{\partial \theta_2}, \cdots, 
\frac{\partial f}{\partial \theta_S} .
$$
The DAG is then made of:
 * *directed* edges: represent the flows of data;
 * the root of the graph (a node which does not has any child): contains the (scalar) result of the computation of $f(\boldsymbol{\theta})$;
 * the leaf nodes (nodes which do not have any parent): contain the variables $(\theta_1, \theta_2, \cdots, \theta_S)$ with repect to which we want to differentiate $f$;
 * the non-leaf nodes: contain an operation to perform on their inputs.

In PyTorch, the leaf nodes are built either automatically when initializing a `torch.nn.Module` (such as a linear layer, in which weights and biases are by-default leaf nodes), or manually trough the class `torch.nn.Parameter`. Tensors that we use during a computation, but that are not variables of our function are just instances of the class `torch.Tensor` (with `requires_grad = False`).

The DAG is built on-the-fly during the successive operations, and the gradients are computed only when calling the `backward` method on the root node, or the function `torch.autograd.grad` on the root node and the leaf nodes with respect to which we want to differentiate. Using `backward` is very straightforward, while `torch.autograd.grad` is slightly more complex, but it supports advanced custom operations.

## Parameters and tensors

**Question 1**

Build some `torch.nn.Parameter`, some `torch.Tensor` and some `torch.Tensor` with `requires_grad = True`.

Note: The main difference between a `Parameter` and a `Tensor` with `requires_grad = True` is how they are managed inside a PyTorch `Module`. To avoid any unexpected behavior, one should prefer to use `Parameter`.

## Computing gradients

**Question 2**

Let $f$ be the function defined below. Compute its derivative with respect to `x1` and `x2` by using `backward`, and then by using `torch.autograd.grad`.

### backward

In [ ]:
x1 = nn.Parameter(torch.randn(5))
x2 = nn.Parameter(torch.randn(1).squeeze())
a = torch.randn(5)
b = torch.randn(1).squeeze()

y = x2 * torch.sin((a * x1).sum() + b)

### torch.autograd.grad

### Higher-order derivatives

**Question 3**

Compute $\frac{\partial f}{\partial x_1 \partial x_2}$ by using `torch.autograd.grad` twice. One check how to use the additional parameters `create_graph` and `allow_unused`.

## Access the computational graph

**Question 4**

The root node contains all the required information to perform the backpropagation and compute the derivatives. The graph and the nodes can be accessed with the methods `grad_fn` and `next_functions`. The leaf nodes can be accessed with the `variable` method. Before the backward, the intermediady results can be accessed with `_saved_self`.

Access the various nodes of the graph. 

In [ ]:
# Other example

x = nn.Parameter(torch.randn(6))
y = torch.split(x, 2)
z = sum(y).sum()
z.backward()

## Forward-only mode

**Question 5**

Sometimes, for instance when testing a model, we are just interested in the result, and not the gradients. Such computations are more efficient if we are in a "forward-only" mode (in that situation, the computational graph is not created and the intermediary results are not stored).

The easiest way is to use `with torch.no_grad()`. One can also use the method `detach` on the parameters to use their content without building the computational graph.

Compute the function $f$ without building the computational graph using both methods and check that no computational graph is stored.

In [ ]:
# with torch.no_grad()



In [ ]:
# detach()



# Managing a dataset

Before training a model on a dataset, we have to make some basic checks on the dataset and preprocess it correctly in order to obtain the best possible results (on a validation set).

In [ ]:
datasets_path = "" # ...

## Checking a sample

First, we load MNIST.

In [ ]:
batch_size = 64

# build transform
transform = transforms.Compose([
    transforms.ToTensor(),
    ]) 

# choose the training and test datasets
train_data = datasets.MNIST(datasets_path, train = True,
                              download = False, transform = transform)
test_data = datasets.MNIST(datasets_path, train=False,
                             download = False, transform = transform)

train_size = len(train_data)
test_size = len(test_data)

# build the data loaders
train_loader = torch.utils.data.DataLoader(train_data, batch_size = batch_size)
test_loader = torch.utils.data.DataLoader(test_data, batch_size = batch_size, shuffle = False)

# specify the image classes
classes = [f"{i}" for i in range(10)]

**Question 1**

By using `numpy.random.choice`, subplots and the function `imshow` of matplotlib, print 10 (or more) random data points of the training set (image + label).

Is the task feasible for a human? Do the data look clean ?

## Check for class imbalance

**Question 2**

In classification tasks, it is common that the different classes are inequally represented in the dataset. If this "class imbalance" is too severe, the model is likely to fail to learn well the least represented classes.

Compute the number of data points in each class (in the training dataset). What do we observe?

Theoretical number of data points per class if the dataset was perfectly balanced: 6000.

# Managing a model

In [ ]:
class LeNet(nn.Module):
    def __init__(self):
        super(LeNet, self).__init__()

        self.act_function = torch.tanh
        layers = [1, 6, 16, 120, 84, 10]

        self.conv1 = torch.nn.Conv2d(layers[0], layers[1], 5, padding = 2)
        self.conv2 = torch.nn.Conv2d(layers[1], layers[2], 5)
        self.fc1 = torch.nn.Linear(5 * 5 * layers[2], layers[3])
        self.fc2 = torch.nn.Linear(layers[3], layers[4])
        self.fc3 = torch.nn.Linear(layers[4], layers[5])


    def forward(self, x):
        x = self.conv1(x)
        x = self.act_function(x)
        x = torch.nn.functional.max_pool2d(x, 2)

        x = self.conv2(x)
        x = self.act_function(x)
        x = torch.nn.functional.max_pool2d(x, 2)

        x = x.view(x.size(0), -1)

        x = self.fc1(x)
        x = self.act_function(x)

        x = self.fc2(x)
        x = self.act_function(x)

        x = self.fc3(x)
        x = torch.nn.functional.log_softmax(x, dim = 1)

        return x

**Question 1**

When a `torch.nn.Module` is created, its parameters and its submodules are automatically registered, which is necessary to optimize them with an `Optimizer`. One can access them with the methods `named_parameters`, `parameters`, `named_modules`, `modules`.

Access the various elements on an instance of LeNet.

**Question 2**

Create an instance of the class below and print its modules and parameters. What do we observe? Fix this issue by using the object `torch.nn.ModuleList`.

In [ ]:
class SomeModel(nn.Module):
    def __init__(self):
        super(SomeModel, self).__init__()

        self.layers = [torch.nn.Linear(5, 5),
                      torch.nn.ReLU(),
                      torch.nn.Linear(5, 1)]

    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
        return x

In [ ]:
class SomeModelFixed(nn.Module):
    def __init__(self):
        pass
        # ...

    def forward(self, x):
        pass
        # ...

**Question 3 (optional)**

Check out the method `register_buffer` of `Module`, explain why it can be useful and show a use-case.

# Training a model

## Basic training

**Question 1**

We want to train LeNet on MNIST. Fill the blanks in the following pieces of code.

Wrap the training process into a function.

In [ ]:
# model
model = # ...

# loss function
criterion = nn.CrossEntropyLoss()

# optimizer
optimizer = optim.SGD(model.parameters(), lr = .01)

In [ ]:
nepochs = 50

#List to store loss to visualize
valid_loss_min = np.inf # track change in validation loss

train_losses = []
test_losses = []
acc_eval = []
#test_counter = [i*len(train_loader.dataset) for i in n_epochs]

for epoch in range(nepochs):
    # keep track of training and validation loss
    train_loss = 0.
    valid_loss = 0.
    
    ###################
    # train the model #
    ###################
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        # ...
        
    ######################    
    # validate the model #
    ######################
    model.eval()
    correct = 0
    for data, target in test_loader:
        # ...
    
    # calculate average losses
    train_loss = train_loss/len(train_loader.dataset)
    valid_loss = valid_loss/len(test_loader.dataset)
    acc_eval.append(correct/len(test_loader.dataset)*100)
    train_losses.append(train_loss)
    test_losses.append(valid_loss)
    
    # print training/validation statistics 
    print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(
        epoch, train_loss, valid_loss))

## Data normalization

**Question 2**

To make sure that the inputs of the neural network are within a controlled range, we usually transform the dataset to be sure that the data are centered with variance 1. It is not always necessary, but it is worth knowing it.

Check the range of values on a sample of MNIST. Compute the mean and the standard deviation of the training dataset of MNIST and normalize the dataset accordingly by using `transforms.Normalize`.

## Data augmentation

**Question 3**

It is very common to face overfitting when doing deep learning. So, several methods can be used to solve this problem. One of them is called "data augmentation". It consists in adding "noise" to data points of the training dataset in order to make the model resistant to small changes of the data. 

When training on images, it is common to perform "random crops", "random flips", and small "random rotations". With MNIST, it is meaningless to add random flips, because most digits are not supposed to be invariant by vertical or horizontal symmetries.

Add random crops with `transforms.RandomCrop` with a reasonable number of pixels to the transforms to do on the dataset, visualize the resulting images and train the model.

## Influence of the initialization

**Question 4**

Build a Multilayer Perceptron with ReLU activation functions, which takes 3 arguments: 
 * `layers`: the list of layer sizes;
 * `sigma_w`: the standard deviation chosen for initializing the weights;
 * `with_scaling`: if True, we multiply the generated weights by $1/\sqrt{\# \text{inputs}}$.

Write the `reset_parameters` method, which initialize the weights according to a Gaussian distribution, either with variance $\sigma_w^2$, or $\sigma_w^2/\# \text{inputs}$.

In [ ]:
class Perceptron(torch.nn.Module):
    def __init__(self, layers, sigma_w, scaling = False):
        super(Perceptron, self).__init__()

        # ...

        self.reset_parameters()

    def reset_parameters(self):
        pass
        # ...

    def forward(self, x):
        pass
        # ...

**Question 4**

Train the model with various choices of initialization (which ones?) and try various numbers of layers with various widths.

**Question 5 (optional)**

Implement the NTK parameterization in the `Perceptron` model: divide by $1/\sqrt{\# \text{inputs}}$ the result of each layer.

Train such a network with the SGD (with `scaling = False`) with learning rates of order $1$.

# Adam

**Question 6**

Train LeNet with the SGD and Adam with default parameters and compare.

## Other stuff

**Additional questions**

 * explore data augmentation with the dataset CIFAR-10
 * test different batch sizes. How to change the learning rate when we change the batch size?
 * test drop-out layers
 * read the documentation on SGD and Adam an propose a way to assign different learning rates to different sets of parameters